In [ ]:
import numpy as np 
import pandas as pd
import tensorflow.keras as tf
import tensorflow_addons as tfa
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
K_FOLDS = 5

In [ ]:
train_features = pd.read_csv("/kaggle/input/lish-moa/train_features.csv")
train_labels = pd.read_csv("/kaggle/input/lish-moa/train_targets_scored.csv")
test_features = pd.read_csv("/kaggle/input/lish-moa/test_features.csv")

# **Preparing a dataset**

In [ ]:
df = pd.DataFrame(train_features)
dl = pd.DataFrame(train_labels)

In [ ]:
def DataPreProcessing(dataframe):
    dataframe['cp_type'] = dataframe['cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    dataframe['cp_time'] = dataframe['cp_time'].map({ 24: 0, 48: 0.5, 72: 1})
    dataframe['cp_dose'] = dataframe['cp_dose'].map({'D1': 0, 'D2': 1})

In [ ]:
DataPreProcessing(df)

In [ ]:
rf = df[df["cp_type"] == 1]["sig_id"]

df = df[~df["sig_id"].isin(rf)]
dl = dl[~dl["sig_id"].isin(rf)]

In [ ]:
df

In [ ]:
dl

In [ ]:
df = df.drop(["sig_id","cp_type"], axis=1)
dl = dl.drop(["sig_id"], axis=1) #, "atp-sensitive_potassium_channel_antagonist","erbb2_inhibitor"], axis=1)

In [ ]:
data_features = df.values
data_labels = dl.values

fold = int(len(data_features) / K_FOLDS)

# **Building a model**

In [ ]:
def NN_Model(model, fold, k, X, y):
    
    i = k*fold
    j = (k+1)*fold
    
    v_features = X[i:j]
    v_labels = y[i:j]
    
    X_features = np.delete(X, slice(i,j), axis=0)
    y_labels = np.delete(y, slice(i,j), axis=0)
    
    
    model.add(tf.layers.BatchNormalization())
    

    model.add(tfa.layers.WeightNormalization(tf.layers.Dense(876, activation='relu')))
    model.add(tf.layers.BatchNormalization())
    model.add(tf.layers.Dropout(0.7))

    
    model.add(tfa.layers.WeightNormalization(tf.layers.Dense(876, activation='relu')))
    model.add(tf.layers.BatchNormalization())
    model.add(tf.layers.Dropout(0.6))

    model.add(tfa.layers.WeightNormalization(tf.layers.Dense(512, activation='relu')))
    model.add(tf.layers.BatchNormalization())

    
    model.add(tf.layers.Dense(206, activation='sigmoid'))
    
    model.compile(optimizer='adam'
                  ,loss='binary_crossentropy'
                 )

    #model.summary()

    res = model.fit(X_features, y_labels, epochs=22, validation_data=(v_features, v_labels))

In [ ]:
fold

In [ ]:
# test
#model = tf.models.Sequential()
#NN_Model(model, fold, 0, data_features, data_labels)

In [ ]:
model_list = []

for f in range(0, K_FOLDS):
    model = tf.models.Sequential()
    NN_Model(model, fold, f, data_features, data_labels)
    model_list.append(model)
    print("")

**Samples**

In [ ]:
tf = pd.DataFrame(test_features)
DataPreProcessing(tf)

test_data_features = tf.drop(["sig_id","cp_type"], axis=1).values

In [ ]:
p = model_list[0].predict(test_data_features)

for f in range(1, K_FOLDS):
    p += model_list[f].predict(test_data_features)

In [ ]:
prediction = p / K_FOLDS

In [ ]:
res_set = pd.DataFrame.from_records(prediction)
res_set.insert(0,'cp_type',test_features['cp_type'])

res_set.loc[res_set['cp_type'] == 1, res_set.columns != "cp_type"] = 0

res_set = res_set.drop(["cp_type"], axis=1)
res_set.insert(0,'sig_id',test_features['sig_id'])

#res_set.insert(35,'atp-sensitive_potassium_channel_antagonist',0)
#res_set.insert(83,'erbb2_inhibitor',0)

res_set.columns = train_labels.columns

#res_set

res_set.to_csv('submission.csv', index=False)